# Texture feature value simulator
Copyright (c) 2016 Tetsuya Shinaji

This software and figures are released under the MIT License.

http://opensource.org/licenses/mit-license.php

In [31]:
import numpy as np
from matplotlib import pyplot as plt
import sys
sys.path.append('../')
from TextureAnalysis import GLCM_3D, GLSZM_3D, NGTDM, GLHA
from ipywidgets import interact
from IPython.core.pylabtools import figsize
import pandas as pd
from IPython.display import display
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

def get_circle_img(r):
    y, x = np.ogrid[-64:128-64, -64:128-64]
    mask = x*x + y*y <= r*r
    array = np.zeros((1, 128, 128))
    array[0, mask] = 1
    return array, np.sqrt(x*x + y*y)

%matplotlib inline

## GLCM: Gray-Level Co-occurrence Matrix

![image](./imgs/GLCM.png)

In [17]:
figsize(10, 10)
@interact(max_intensity=(1, 64, 1), heterogeneity=(1, 100, 1), intensity_variation=(0, 100, 1), size=(1, 64, 1))
def test_glcm_dist(max_intensity=50, heterogeneity=15, intensity_variation=85, size=35):
    base_img, distance = get_circle_img(size)
    if intensity_variation == 0:
        grad_img = np.ones_like(base_img)
    else:
        grad_img = np.exp(-distance/(101-intensity_variation)).reshape(1, 128, 128)
    example_img =  np.random.normal(base_img*max_intensity*grad_img, heterogeneity/2.354820)*base_img.astype(float)
    glcm = GLCM_3D(example_img, d=1, level_min=1, level_max=64, threshold=1)
    plt.subplot(121)
    plt.subplots_adjust(wspace=0.35)
    plt.title('A given image')
    img = plt.imshow(glcm.img[0], vmin=0)
    cbar = plt.colorbar(img, fraction=0.046, pad=0.04)
    cbar.set_label('Pixel value')
    plt.subplot(122)
    plt.title('GLCM')
    img = plt.imshow(glcm.matrix*100, extent=([1, 64, 64, 1]))
    plt.ylabel('Center pixel value')
    plt.xlabel('Neighbor pixel value')
    cbar = plt.colorbar(img, fraction=0.046, pad=0.04)
    cbar.set_label('Probability [%]')
    plt.show()
    features =pd.DataFrame.from_dict(glcm.features, orient="index")
    features.columns = ['Texture feature value']
    display(features)

## GLSZM: Gray Level Size Zone Matrix

![image](./imgs/GLSZM.png)

In [18]:
figsize(10, 10)
@interact(max_intensity=(1, 64, 1), heterogeneity=(1, 100, 1), intensity_variation=(0, 100, 1), size=(1, 64, 1))
def test_normal_dist(max_intensity=50, heterogeneity=15, intensity_variation=85, size=35):
    base_img, distance = get_circle_img(size)
    if intensity_variation == 0:
        grad_img = np.ones_like(base_img)
    else:
        grad_img = np.exp(-distance/(101-intensity_variation)).reshape(1, 128, 128)
    example_img =  np.random.normal(base_img*max_intensity*grad_img, heterogeneity/2.354820)*base_img.astype(float)
    glszm = GLSZM_3D(example_img, level_min=1, level_max=64, threshold=1)
    plt.subplots_adjust(wspace=0.5)
    fig = plt.gcf()
    ax1 = plt.subplot(121)
    ax1.set_title('A given image')
    img1 = ax1.imshow(glszm.img[0], vmin=0)
    cbar1 = fig.colorbar(img1,fraction=0.046, pad=0.04)
    cbar1.set_label('Pixel value')
    ax2 = plt.subplot(122)
    ax2.set_title('GLSZM')
    img2 = ax2.imshow(glszm.matrix, 
                     aspect=glszm.matrix.shape[1]/glszm.matrix.shape[0],
                     extent=[0.5, glszm.matrix.shape[1]+0.5, glszm.matrix.shape[0]+1, 1])
    ax2.set_ylabel('Pixel value')
    ax2.set_xlabel('Size')
    cbar2 = fig.colorbar(img2, fraction=0.046, pad=0.04)
    cbar2.set_label('Number of areas')
    plt.show()
    features =pd.DataFrame.from_dict(glszm.features, orient="index")
    features.columns = ['Texture feature value']
    display(features)

## NGTDM: Neighbourhood Gray-Tone-Difference Matrix

![image](./imgs/NGTDM.png)

In [4]:
figsize(10, 10)
@interact(max_intensity=(1, 64, 1), heterogeneity=(1, 100, 1), intensity_variation=(0, 100, 1), size=(1, 64, 1))
def test_normal_dist(max_intensity=50, heterogeneity=15, intensity_variation=85, size=35):
    base_img, distance = get_circle_img(size)
    if intensity_variation == 0:
        grad_img = np.ones_like(base_img)
    else:
        grad_img = np.exp(-distance/(101-intensity_variation)).reshape(1, 128, 128)
    example_img =  np.random.normal(base_img*max_intensity*grad_img, heterogeneity/2.354820)*base_img.astype(float)
    ngtdm = NGTDM(example_img[0], level_min=0, level_max=63, threshold=1)
    plt.subplots_adjust(wspace=0.5)
    fig = plt.gcf()
    ax1 = plt.subplot(121)
    ax1.set_title('A given image')
    img1 = ax1.imshow(ngtdm.img, vmin=0)
    cbar1 = fig.colorbar(img1,fraction=0.046, pad=0.04)
    cbar1.set_label('Pixel value')
    ax2 = plt.subplot(122)
    ax2.set_title('NGTDM')
    img2 = ax2.imshow(ngtdm.s.reshape(ngtdm.s.size, 1), 
                     aspect=1/ngtdm.s.shape[0],
                     extent=[0.5, 1+0.5, ngtdm.s.shape[0]+1, 1])
    ax2.set_ylabel('Pixel value')
    cbar2 = fig.colorbar(img2, fraction=0.046, pad=0.04)
    plt.show()
    features =pd.DataFrame.from_dict(ngtdm.features, orient="index")
    features.columns = ['Texture feature value']
    display(features)

## GLHA: Gray Level Histogram Analysis

In [44]:
figsize(10, 10)
@interact(max_intensity=(1, 64, 1), heterogeneity=(1, 100, 1), intensity_variation=(0, 100, 1), size=(1, 64, 1))
def test_normal_dist(max_intensity=50, heterogeneity=15, intensity_variation=85, size=35):
    base_img, distance = get_circle_img(size)
    if intensity_variation == 0:
        grad_img = np.ones_like(base_img)
    else:
        grad_img = np.exp(-distance/(101-intensity_variation)).reshape(1, 128, 128)
    example_img =  np.random.normal(base_img*max_intensity*grad_img, heterogeneity/2.354820)*base_img.astype(float)
    glha = GLHA(example_img, level_min=0, level_max=63, threshold=1)
    plt.subplots_adjust(wspace=0.5)
    fig = plt.gcf()
    ax1 = plt.subplot(121)
    ax1.set_title('A given image')
    img1 = ax1.imshow(glha.img[0], vmin=0)
    cbar1 = fig.colorbar(img1, fraction=0.046, pad=0.04)
    cbar1.set_label('Pixel value')
    ax2 = plt.subplot(122)
    hst = plt.hist(glha.img.flatten(), bins=64, range=[1, 64])
    plt.xlabel('Pixel value')
    plt.ylabel('Frequency')
    plt.title('Histogram')
    ax2.set_aspect(64/hst[0].max())
    plt.show()
    features =pd.DataFrame.from_dict(glha.features, orient="index")
    features.columns = ['Texture feature value']
    display(features)